# Polygons

## The Problem

Consider an n-sided polygon where n is even. We draw all the lines that connect the vertices, and mark out the intersection points. This creates a graph with the vertices and intersection points as the nodes of the graph, and the edges are the line segments between the vertices and the intersection points. You can refer to the plot below.

We define a start point and an end point as the opposite vertices of the polygon. We also define a "decreasing path" as a path along the edges of the graph that links the start point to the end point, where the distance to the end point is strictly decreasing while moving along the path. Find the total number of decreasing paths available.

## The Solution

In [ ]:
import numpy as np
import pandas as pd
import itertools
import operator

from decimal import *
from decimal_math import sin, cos
from math import pi, sqrt, comb
from typing import List, Tuple, Set, Dict
from functools import cached_property, lru_cache
from matplotlib import pyplot as plt
from tqdm.auto import tqdm

### Create the regular polygon
```PolygonSolver.__init__()```

Let's first create the regular polygon.
1. Centred on (0,0) with radius 1 unit; thus start point is (1,0) and end point is (-1,0)
2. Use the internal angle of a polygon to find the points of the vertices

Some of the code for Line and Point classes (on the implementation of Cramer's rule) are adapted from https://stackoverflow.com/a/20679579. Also written with mathematical reference to https://observablehq.com/@toja/line-box-intersection.

### Find the intersection points
Next, we draw a line segment for every pair of polygon vertices. We then find the intersection points of every pair of line segments.

### Get all the line segments
Every line segment in the diagram above fulfills the following conditions:
1. Both ends of the line segment must be either an intersection point or a vertex
2. They must be an equal or shorter line deriving from one of the already drawn lines

A possible approach would be to:
1. For every line segment that we have generated, first find which points/vertices lie on it.
2. Break up the line segment into smaller line segments where possible
    - Sort the points by x, then within that by y.
    - Line segments are built by going across the list and taking a pair of points as line segments

### Label all the line segments
Due to the "decreasing path" constraint, there can only be one direction of movement for an edge. We can say that every edge in the graph must be directed. Right angles are particularly helpful in determining where the edge directions point towards.

### Solving the graph

A possible approach would be to find the value of the number of paths to reach a node from the start point. We can solve this recursively as the value of any node in the graph is the sum of the value of the points that it is dependent on.

In [ ]:
getcontext()

In [ ]:
TOLERANCE = Decimal(1e-10)
PRECISION = 10

def zeroise(a: float or None) -> bool:
    if a is None:
        return None
    return a if abs(a) > TOLERANCE else 0

In [ ]:
class Point:
    
    def __init__(self, x: float, y: float, precision=PRECISION):
        self.x = round(zeroise(Decimal(x)), precision)
        self.y = round(zeroise(Decimal(y)), precision)
        
    @classmethod
    def from_intersection(cls, L1:"LineSegment"=None, L2:"LineSegment"=None) -> "Point":
        '''
        Alternate constructor for Point via intersection of two LineSegments.
        '''
        a1, b1, c1 = (L1.p2.x - L1.p1.x), (L2.p1.x - L2.p2.x), (L2.p1.x - L1.p1.x)
        a2, b2, c2 = (L1.p2.y - L1.p1.y), (L2.p1.y - L2.p2.y), (L2.p1.y - L1.p1.y)
        
        D  = a1 * b2 - b1 * a2
        Dx = c1 * b2 - b1 * c2
        Dy = a1 * c2 - c1 * a2
        
        if D != 0:
            s = Dx / D
            t = Dy / D
            
            if not (0 <= s <= 1 and 0 <= t <= 1):
                return None
        else:
            return None
        
        return cls((1-s)*L1.p1.x + s*L1.p2.x, (1-s)*L1.p1.y + s*L1.p2.y)

    def reduced_precision(self, precision=PRECISION) -> "Point":
        x = round(zeroise(Decimal(self.x)), precision)
        y = round(zeroise(Decimal(self.y)), precision)
        return Point(x, y)
    
    def connecting_edges(self, polygon_solver) -> Set["LineSegment"]:
        '''
        Method by which the point becomes aware of its adjacent edges.
        Can be created at intersection time in the future.
        '''
        return polygon_solver.point_ls[self]
    
    @lru_cache(maxsize=None)
    def dependent_edges(self, polygon_solver) -> Set["LineSegment"]:
        '''
        Provides the edges that we depend on to calculate self.value.
        In approach one, these are the edges that point towards self.
        '''
        return set(edge for edge in self.connecting_edges(polygon_solver) if edge.direction(self))
        
    @lru_cache(maxsize=None)
    def value(self, polygon_solver) -> int:
        '''
        Recursively calculates the number of paths we can take from the start node to this node by summing over
            the nodes of dependent edges.
        '''
        if not self.dependent_edges(polygon_solver):
            return 1
        return sum(edge.other_end(self).value(polygon_solver) for edge in self.dependent_edges(polygon_solver))
    
    def __eq__(self, other) -> bool:
        if isinstance(other, Point):
            return not zeroise(self.x - other.x) and not zeroise(self.y - other.y)
        raise TypeError(f'Unable to compare Point and {other}.')

    def __hash__(self):
        return hash((self.x, self.y))
    
    def __repr__(self) -> str:
        return f"Point({self.x}, {self.y})"

In [ ]:
class LineSegment:
    def __init__(self, p1: "Point", p2: "Point"):
        self.p1, self.p2 = sorted([p1, p2], key=operator.attrgetter('x','y'))
        
    @cached_property
    def L2(self) -> float:
        return sqrt((self.p1.x - self.p2.x)**2 + (self.p1.y - self.p2.y)**2)
    
   
    @cached_property
    def mid(self) -> "Point":
        return Point((self.p1.x + self.p2.x)/2, (self.p1.y + self.p2.y)/2)
    
    def direction(self, pt:"Point"=None) -> bool:
        '''
        Provides the direction label for the graph, by comparing the L2 distances (p1-to-end, p2-to-end).
            True if p1-to-p2; False if p2-to-p1.
        
        If the argument <pt> is provided, then returns True if the direction is pointing towards <pt> (which
            should be one of the ends of the line segment).
        
        Caution: This method should only be used when there are no intersection points between self.p1 and self.p2.
            Otherwise, it does not return a useful directional label.
        '''
        direction = LineSegment(self.p1, END).L2 > LineSegment(self.p2, END).L2
        if pt is None:
            return direction
        else:
            return direction if pt == self.p2 else (not direction)

    @cached_property
    def gradient(self) -> Tuple[float, float]:
        '''
        Creates a tuple used in the visualisation of the direction.
        '''
        p1, p2 = self.p1, self.p2
        multiplier = 1 if self.direction() else -1
        x_coeff = 0 if p1.x == p2.x else 1
        delta = Decimal(0.01)
        
        if p2.y > p1.y:
            y_coeff = 1
        elif p1.y == p2.y:
            y_coeff = 0
        else:
            y_coeff = -1
        
        try:
            return (multiplier*x_coeff*delta, multiplier*y_coeff*delta*abs((p2.y - p1.y)/(p2.x - p1.x)))
        except (DivisionByZero, InvalidOperation):
            return (0, multiplier*delta)
    
    def other_end(self, p: "Point") -> "Point":
        '''
        Returns the other end of the line if one end is given, None otherwise.
        '''
        if p == self.p1:
            return self.p2
        elif p == self.p2:
            return self.p1
        else:
            return None
        
    def __eq__(self, other) -> bool:
        if isinstance(other, LineSegment):
            return self.p1 == other.p1 and self.p2 == other.p2
        return False
    
    def __hash__(self):
        return hash((self.p1, self.p2))
    
    def __repr__(self) -> str:
        return f"LineSegment({self.p1}, {self.p2})"    

In [ ]:
END = Point(-1,0)
START = Point(1,0)

In [ ]:
class PolygonSolver:
    def __init__(self, n, plot=False, **kwargs):
        '''
        Creating an instance will create the regular polygon.
        '''
        self.n = n
        self.polygon = self.create_regular_polygon(n)
        
        self.line_segments = None
        self.point_ls = None
        self.create_line_segments()
        
        if plot:
            self.plot_polygon_graph(**kwargs)

    @classmethod
    def find_next_coordinate(self, xy: Tuple[Decimal, Decimal], t: Decimal) -> Tuple[Decimal, Decimal]:
        '''
        Refer to https://en.wikipedia.org/wiki/Rotation_matrix for the mathematical grounding behind this.
        '''
        x, y = xy
        return Point(x*cos(t) - y*sin(t), x*sin(t) + y*cos(t), 25)
            
    @classmethod
    def create_regular_polygon(self, n: int) -> List[Tuple]:
        '''
        Defines a polygon as the List of (x,y) coordinates of the vertices.
        '''
        theta = Decimal(2/n * pi)
        polygon = []
        previous = START

        for i in range(n):
            polygon.append(previous)
            previous = self.find_next_coordinate((previous.x, previous.y), theta)
            
        return polygon
    
    @cached_property
    def lines(self) -> List[LineSegment]:
        return [LineSegment(a,b) for a,b in itertools.combinations(self.polygon, 2)]
    
    @cached_property
    def line2points(self) -> Dict[LineSegment, Set[Point]]:
        '''
        Contains the information of where each line is mapped to every point on it.
        '''
        line2points = dict()  
        for line in self.lines:
            line2points[line] = set([line.p1, line.p2])

        for a, b in itertools.combinations(self.lines, 2):
            point = Point.from_intersection(a,b)

            if point is not None:
                line2points[a].add(point)
                line2points[b].add(point)

        return line2points
    
    def create_line_segments(self) -> None:
        line_segments = set()
        point_ls = dict()
        
        for ls, p in self.line2points.items():
            if len(p) == 2:
                line_segments.add(ls)
            else:
                sorted_points = sorted([item.reduced_precision() for item in p], key=operator.attrgetter('x','y'))
                for previous, current in zip(sorted_points, sorted_points[1:]):
                    if previous != current:
                        edge = LineSegment(previous, current)
                        line_segments.add(edge)
                        for item in (previous, current):
                            if item not in point_ls:
                                point_ls[item] = set([edge])
                            else:
                                point_ls[item].add(edge)
                                
        self.line_segments = line_segments
        self.point_ls = point_ls

    def plot_polygon_graph(self, figsize:int=8, lines:bool=True, points:bool=True, values:bool=True,
                           point_size:float=1, head_width:float=0.03):
        x, y = [point.x for point in self.point_ls], [point.y for point in self.point_ls]

        # Configurations
        limit = 1.1
        head_width = head_width
        offset = -0.07

        fig, ax = plt.subplots(figsize=(figsize,figsize))
        ax.set_xlim(-limit, limit)
        ax.set_ylim(-limit, limit)
        ax.set_aspect('equal', adjustable='box')
        ax.grid(axis='both', alpha=0.2)

        # Lines
        if lines:
            for line in self.line_segments:
                color = next(ax._get_lines.prop_cycler)['color']
                ax.plot([line.p1.x, line.p2.x], [line.p1.y, line.p2.y], color=color)
                plt.arrow(x=line.mid.x, y=line.mid.y,
                          dx=line.gradient[0], dy=line.gradient[1],
                          lw=0, color=color, length_includes_head=True, head_width=head_width)

        # Points
        if points:
            ax.scatter(x, y, c="black", s=point_size)

        # Values
        if values:
            for pt in self.point_ls:
                plt.annotate(pt.value(self), (pt.x+offset, pt.y+offset))
    
    def check_intersections(self):
        '''
        Returns the total number of intersections for an even regular n-sided polygon.
        Formula from <https://www.math.uwaterloo.ca/~mrubinst/publications/ngon.pdf>
        '''
        def is_multiple(x: int, y: int) -> bool:
            return x % y == 0

        n = self.n
        d = is_multiple
        interior = (comb(n, 4) + (-5*n**3 + 45*n**2 - 70*n + 24)/24*d(n,2) - (3*n/2)*d(n,4) + (-45*n**2 + 262*n)/6*d(n,6)
                    + 42*n*d(n,12) + 60*n*d(n,18) + 35*n*d(n,24) - 38*n*d(n,30) - 82*n*d(n,42) - 330*n*d(n,60)
                    - 144*n*d(n, 84) - 96*n*d(n,90) - 144*n*d(n,120) - 96*n*d(n,210))
        
        try:
            assert self.n + interior == len(self.point_ls)
            print(f'n = {n}, total number of points = {len(self.point_ls)}')
        except AssertionError:
            diff = len(self.point_ls) - int(self.n) - int(interior)
            raise AssertionError(f'Theoretical number of points ({int(self.n)} + {int(interior)} = {int(self.n + interior)}) not equal to calculated number of points ({len(self.point_ls)}). Difference of {diff}.')
    
    @lru_cache(maxsize=None)
    def solve(self):
        return sorted(self.point_ls.keys(), key=operator.attrgetter('x', 'y'))[0].value(self)
    
    @lru_cache(maxsize=None)
    def progressive_solve(self):
        '''
        Intended as an alternative solver that allows us to progressively solve the graph.
        Allows us to see a progress bar with a marginal performance hit compared to PolygonSolver.solve().
        '''
        iterable = sorted(self.point_ls.keys(), key=operator.attrgetter('x', 'y'), reverse=True)
        for item in tqdm(iterable, total=len(iterable)):
            item.value(self)
        return iterable[-1].value(self)

In [ ]:
%%time
# Solver - plotting is slow and not recommended for n>20
solver = PolygonSolver(n=78, plot=False, figsize=20, values=False, head_width=0.01, point_size=12)
solver.check_intersections()
solver.progressive_solve()

### Debugging
#### Assertion errors
The following code below is used for debugging purposes when the assertion does not hold.

In [ ]:
all_points = sorted(list(solver.point_ls.keys()), key=operator.attrgetter('x', 'y'))

In [ ]:
def sliding_window_iter(iterable, size):
    # https://codereview.stackexchange.com/questions/239352/sliding-window-iteration-in-python
    iterables = itertools.tee(iter(iterable), size)
    window = zip(*(itertools.islice(t, n, None) for n,t in enumerate(iterables)))
    yield from window

In [ ]:
similar_points = set()
for item in tqdm(sliding_window_iter(all_points, 5), total=len(all_points)):
    for a,b in itertools.combinations(item,2):
        if abs(a.x - b.x) < TOLERANCE and abs(a.y - b.y) < TOLERANCE:
            similar_points.add((a,b))
similar_points = list(similar_points)

In [ ]:
similar_points

#### Recursion errors
Recursion errors when a node is connected to itself. To debug, we need to find a node that is connected to itself. The connection may not be immediate; it could be connected to another node; which then links to another node, and then to itself.

In [ ]:
recursive_points = []
for point in tqdm(solver.point_ls, total=len(solver.point_ls)):
    for edge in point.dependent_edges(solver):
        if edge.other_end(point) == point:
            recursive_points.append(point)

In [ ]:
recursive_points

In [ ]:
problematic_pt = sorted(solver.point_ls.keys(), key=operator.attrgetter('x', 'y'), reverse=True)[661774]

In [ ]:
problematic_pt

In [ ]:
# list(problematic_pt.dependent_edges(solver))[1].other_end(problematic_pt).value(solver)

### Batch run

In [ ]:
current_values = dict()

In [ ]:
def batch_solver(left_bound, right_bound):
    for n in range(left_bound, right_bound, 2):
        solver = PolygonSolver(n=n, plot=False, figsize=20, values=False, head_width=0.01, point_size=12)
        solver.check_intersections()
        current_values[n] = solver.progressive_solve()

In [ ]:
batch_solver(80,100)

### Computational time
We are currently running into problems where we take a long time to compute.

There are a few ways to improve this algorithm.
1. Exploit the symmetry in the graph along the x-axis to realise that for any point, $(x_1,y_1).value$ will be the same as $(x_1, -y_1).value$ and thus the calculation only needs to be done once.

In [ ]:
from results import solved_values
a,b = zip(*solved_values.items())

data = {
    'n': a,
    'paths': b
}

df = pd.DataFrame.from_dict(data)
df.set_index('n', drop=True, inplace=True)

In [ ]:
df['ln_paths'] = np.log(df['paths'])
df['cubert_ln_paths'] = np.cbrt(df['ln_paths'])

df.drop('paths', inplace=True, axis=1)
df.drop('ln_paths', inplace=True, axis=1)
df.plot()

As the plot is close to linear, we think that the growth rate in $n$ (where $n$ is the number of sides of the polygon) is $e^{n^3}$.